<a href="https://colab.research.google.com/github/SandeepUpadhyaya/Machine-Learning-Projects/blob/main/Olist(Lifetime_value_prediction)_clustering_and_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Identifying potential customer based from Brazilian E-Commerce Public Dataset

From the above said dataset there were 8 different CSV files was found pertaining customer data, order data, order items, order payments, order reviews, product data, seller data and category name.

In [3]:
# importing libraries
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from __future__ import division
import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

# Products

In [4]:
# replace the product categorie names to english
path1 = "/content/drive/MyDrive/Colab Notebooks/Olist/Olist-drive/olist_products_dataset.csv"
path2 = "/content/drive/MyDrive/Colab Notebooks/Olist/Olist-drive/product_category_name_translation.csv"
products = pd.read_csv(path1)
prod_cat_name_tr = pd.read_csv(path2)
products = products.merge(prod_cat_name_tr, on='product_category_name', how='left')
products.drop ('product_category_name', axis=1, inplace=True)
products.head()

,product_id,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,art
2,96bd76ec8810374ed1b65e291975717f,46.0,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,27.0,261.0,1.0,371.0,26.0,4.0,26.0,baby
4,9dc1a7de274444849c219cff195d0b71,37.0,402.0,4.0,625.0,20.0,17.0,13.0,housewares


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
products.isnull().sum()

product_id                         0
product_name_lenght              610
product_description_lenght       610
product_photos_qty               610
product_weight_g                   2
product_length_cm                  2
product_height_cm                  2
product_width_cm                   2
product_category_name_english    623
dtype: int64

Product name, description and category is missing for more than 600 products, the size details of 2 products is missing we can impute dimension details with the median value or we can drop those 2 row

In [6]:
products=products.rename(columns={'product_name_lenght':'product_name_len','product_description_lenght':'product_description_len'})

In [7]:
products[products['product_weight_g'].isnull()]

,product_id,product_name_len,product_description_len,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
8578,09ff539a621711667c43eba6a3bd8466,60.0,865.0,3.0,NaN,NaN,NaN,NaN,baby
18851,5eb564652db742ff8f28759cd8d2652a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
products = products[~(products['product_weight_g'].isnull())]

Lets check the rows where the product name is missing

In [9]:
products[products['product_name_len'].isnull()].head()

,product_id,product_name_len,product_description_len,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
105,a41e356c76fab66334f36de622ecbd3a,NaN,NaN,NaN,650.0,17.0,14.0,12.0,NaN
128,d8dee61c2034d6d075997acef1870e9b,NaN,NaN,NaN,300.0,16.0,7.0,20.0,NaN
145,56139431d72cd51f19eb9f7dae4d1617,NaN,NaN,NaN,200.0,20.0,20.0,20.0,NaN
154,46b48281eb6d663ced748f324108c733,NaN,NaN,NaN,18500.0,41.0,30.0,41.0,NaN
197,5fb61f482620cb672f5e586bb132eae9,NaN,NaN,NaN,300.0,35.0,7.0,12.0,NaN


No useful information can be gathered with just dimension and product id. We need to know what that product is. Lets drop these 610 rows

In [10]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32949 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   product_id                     32949 non-null  object 
 1   product_name_len               32340 non-null  float64
 2   product_description_len        32340 non-null  float64
 3   product_photos_qty             32340 non-null  float64
 4   product_weight_g               32949 non-null  float64
 5   product_length_cm              32949 non-null  float64
 6   product_height_cm              32949 non-null  float64
 7   product_width_cm               32949 non-null  float64
 8   product_category_name_english  32327 non-null  object 
dtypes: float64(7), object(2)
memory usage: 2.5+ MB


# Order

In [44]:
path3 = "/content/drive/MyDrive/Colab Notebooks/Olist/Olist-drive/olist_order_items_dataset.csv"
o_items = pd.read_csv(path3)
o_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [45]:
o_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [46]:
path4 = "/content/drive/MyDrive/Colab Notebooks/Olist/Olist-drive/olist_orders_dataset.csv"
orders = pd.read_csv(path4,parse_dates = ['order_purchase_timestamp'])
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [47]:
orders.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [48]:
orders['order_approved_at'] = orders['order_approved_at'].fillna(orders['order_purchase_timestamp'])
orders['order_delivered_carrier_date'] = orders['order_delivered_carrier_date'].fillna(orders['order_approved_at'])
orders['order_delivered_customer_date'] = orders['order_delivered_customer_date'].fillna(orders['order_estimated_delivery_date'])

In [49]:
orders.isnull().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
dtype: int64

All the missing values were taken care off.

In [50]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99441 non-null  object        
 5   order_delivered_carrier_date   99441 non-null  object        
 6   order_delivered_customer_date  99441 non-null  object        
 7   order_estimated_delivery_date  99441 non-null  object        
dtypes: datetime64[ns](1), object(7)
memory usage: 6.1+ MB


In [51]:
path5 = "/content/drive/MyDrive/Colab Notebooks/Olist/Olist-drive/olist_order_reviews_dataset.csv"
o_reviews = pd.read_csv(path5)
o_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [52]:
o_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [53]:
path6 ="/content/drive/MyDrive/Colab Notebooks/Olist/Olist-drive/olist_order_payments_dataset.csv" 
o_payments = pd.read_csv(path6)
o_payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [54]:
o_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


# Geolocation

In [55]:
path7 = "/content/drive/MyDrive/Colab Notebooks/Olist/Olist-drive/olist_geolocation_dataset.csv"
geoloc = pd.read_csv(path7)
geoloc.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [56]:
geoloc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB


# Seller

In [57]:
path8 = "/content/drive/MyDrive/Colab Notebooks/Olist/Olist-drive/olist_sellers_dataset.csv"
sellers = pd.read_csv(path8)
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [58]:
sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


# Customer

In [59]:
path9 = "/content/drive/MyDrive/Colab Notebooks/Olist/Olist-drive/olist_customers_dataset.csv"
customers = pd.read_csv(path9)
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [60]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [61]:
orders_df = pd.merge(customers, orders, how ='left', on='customer_id')
orders_df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00


In [62]:
orders_df = pd.merge(orders_df,o_items,how ='left', on='order_id')
orders_df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1.0,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,1.0,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,1.0,bd07b66896d6f1494f5b86251848ced7,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,1.0,a5647c44af977b148e0a3a4751a09e2e,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00,1.0,9391a573abe00141c56e38d84d7d5b3b,4a3ca9315b744ce9f8e9374361493884,2018-07-31 10:10:09,230.00,22.25


In [63]:
col = ['customer_unique_id', 'price', 'order_item_id', 'order_purchase_timestamp']
orders = orders_df[col]
orders.head()

,customer_unique_id,price,order_item_id,order_purchase_timestamp
0,861eff4711a542e4b93843c6dd7febb0,124.99,1.0,2017-05-16 15:05:35
1,290c77bc529b7ac935b93aa66c333dc3,289.00,1.0,2018-01-12 20:48:24
2,060e732b5b29e8181a18229c7b0b2b5e,139.94,1.0,2018-05-19 16:07:45
3,259dac757896d24d7702b9acbbff3f3c,149.94,1.0,2018-03-13 16:06:38
4,345ecd01c38d18a9036ed96c73b8d066,230.00,1.0,2018-07-29 09:51:30


In [64]:
#converting the type of Invoice Date Field from string to datetime.
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])

In [65]:
pd.to_datetime('2017-1-1')

Timestamp('2017-01-01 00:00:00')

In [66]:
#create 3months and 6months dataframes
orders_3_month = orders[(orders.order_purchase_timestamp < pd.to_datetime('2017-6-1')) & (orders.order_purchase_timestamp >= pd.to_datetime('2017-3-1'))].reset_index(drop=True)
orders_6_month = orders[(orders.order_purchase_timestamp >= pd.to_datetime('2017-6-1')) & (orders.order_purchase_timestamp  < pd.to_datetime('2017-12-1'))].reset_index(drop=True)

In [67]:
orders_3_month.head()

,customer_unique_id,price,order_item_id,order_purchase_timestamp
0,861eff4711a542e4b93843c6dd7febb0,124.99,1.0,2017-05-16 15:05:35
1,7f3a72e8f988c6e735ba118d54f47458,89.90,1.0,2017-05-11 13:48:47
2,4390ddbb6276a66ff1736a6710205dca,79.90,1.0,2017-04-14 11:24:56
3,2e6a42a9b5cbb0da62988694f18ee295,29.99,1.0,2017-05-15 17:58:25
4,332cf4e83e16004ba7dca932ce82475b,129.60,1.0,2017-04-02 22:15:27


In [68]:
orders_6_month.head()

,customer_unique_id,price,order_item_id,order_purchase_timestamp
0,4c93744516667ad3b8f1fb645a3116a4,259.90,1.0,2017-09-14 18:14:31
1,57b2a98a409812fe9618067b6b8ebe4f,19.90,1.0,2017-11-16 19:29:02
2,2a7745e1ed516b289ed9b29c7d0539a5,25.30,1.0,2017-11-27 17:23:20
3,918dc87cd72cd9f6ed4bd442ed785235,99.00,1.0,2017-09-09 09:54:57
4,3e6fd6b2f0d499456a6a6820a40f2d79,28.99,1.0,2017-11-01 21:54:10


In [69]:
 #create user for assigning clustering
orders_user = pd.DataFrame(orders_3_month['customer_unique_id'].unique())
orders_user.columns = ['customer_unique_id']

In [70]:
#order cluster method
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [71]:
#calculate recency score
orders_max_purchase =orders_3_month.groupby('customer_unique_id').order_purchase_timestamp.max().reset_index()
orders_max_purchase.columns = ['customer_unique_id','MaxPurchaseDate']
orders_max_purchase['Recency'] = (orders_max_purchase['MaxPurchaseDate'].max() - orders_max_purchase['MaxPurchaseDate']).dt.days
orders_user = pd.merge(orders_user, orders_max_purchase[['customer_unique_id','Recency']], on='customer_unique_id')

kmeans = KMeans(n_clusters=4)
kmeans.fit(orders_user[['Recency']])
orders_user['RecencyCluster'] = kmeans.predict(orders_user[['Recency']])

orders_user = order_cluster('RecencyCluster', 'Recency',orders_user,False)

In [72]:
#calcuate frequency score
orders_frequency = orders_3_month.groupby('customer_unique_id').order_purchase_timestamp.count().reset_index()
orders_frequency.columns = ['customer_unique_id','Frequency']
orders_user = pd.merge(orders_user, orders_frequency, on='customer_unique_id')

kmeans = KMeans(n_clusters=4)
kmeans.fit(orders_user[['Frequency']])
orders_user['FrequencyCluster'] = kmeans.predict(orders_user[['Frequency']])

orders_user = order_cluster('FrequencyCluster', 'Frequency',orders_user,True)

In [73]:
#calcuate revenue score
orders_3_month['Revenue'] = orders_3_month['price'] * orders_3_month['order_item_id']
orders_revenue = orders_3_month.groupby('customer_unique_id').Revenue.sum().reset_index()
orders_user = pd.merge(orders_user, orders_revenue, on='customer_unique_id')

kmeans = KMeans(n_clusters=4)
kmeans.fit(orders_user[['Revenue']])
orders_user['RevenueCluster'] = kmeans.predict(orders_user[['Revenue']])
orders_user = order_cluster('RevenueCluster', 'Revenue',orders_user,True)

In [74]:
orders_user.head()

,customer_unique_id,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster
0,861eff4711a542e4b93843c6dd7febb0,15,3,1,0,124.99,0
1,7f3a72e8f988c6e735ba118d54f47458,20,3,1,0,89.90,0
2,2e6a42a9b5cbb0da62988694f18ee295,16,3,1,0,29.99,0
3,fd2d5fdb84e65fa6b54b98b0e2df5645,9,3,1,0,44.90,0
4,8728c766c84eeda24b3e54fe6e632051,1,3,1,0,117.30,0


In [75]:
#calculate revenue and create a new dataframe for it
orders_6_month['Revenue'] = orders_6_month['price'] * orders_6_month['order_item_id']
orders_user_6_month = orders_6_month.groupby('customer_unique_id')['Revenue'].sum().reset_index()
orders_user_6_month.columns = ['customer_unique_id','m6_Revenue']

In [76]:
orders_user_6_month.head()

,customer_unique_id,m6_Revenue
0,0000f6ccb0745a6a4b88665a16c9f078,25.99
1,0004aac84e0df4da2b147fca70cf8255,180.00
2,0006fdc98a402fceb4eb0ee528f6a8d4,13.90
3,00082cbe03e478190aadbea78542e933,79.00
4,000a5ad9c4601d2bbdd9ed765d5213b3,76.99


In [77]:
orders_merge = pd.merge(orders_user, orders_user_6_month, on='customer_unique_id', how='left')
orders_merge = orders_merge.fillna(0)

In [78]:
orders_merge.head()

,customer_unique_id,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,m6_Revenue
0,861eff4711a542e4b93843c6dd7febb0,15,3,1,0,124.99,0,0.0
1,7f3a72e8f988c6e735ba118d54f47458,20,3,1,0,89.90,0,0.0
2,2e6a42a9b5cbb0da62988694f18ee295,16,3,1,0,29.99,0,0.0
3,fd2d5fdb84e65fa6b54b98b0e2df5645,9,3,1,0,44.90,0,0.0
4,8728c766c84eeda24b3e54fe6e632051,1,3,1,0,117.30,0,0.0


In [79]:
#remove outliers
orders_merge = orders_merge[orders_merge['m6_Revenue']<orders_merge['m6_Revenue'].quantile(0.99)]

In [80]:
#creating 3 clusters
kmeans = KMeans(n_clusters=3)
kmeans.fit(orders_merge[['m6_Revenue']])
orders_merge['LTVCluster'] = kmeans.predict(orders_merge[['m6_Revenue']])

#order cluster number based on LTV
orders_merge = order_cluster('LTVCluster', 'm6_Revenue',orders_merge,True)

#creatinga new cluster dataframe
orders_cluster = orders_merge.copy()

#see details of the clusters
orders_cluster.groupby('LTVCluster')['m6_Revenue'].describe()

,count,mean,std,min,25%,50%,75%,max
LTVCluster,,,,,,,,
0,8452.0,0.007508,0.295046,0.00,0.0000,0.0,0.00,13.99
1,36.0,32.378611,11.175493,16.30,20.7475,34.9,39.99,49.95
2,27.0,72.509259,11.796669,56.99,59.9000,72.0,82.60,93.00


In [81]:
#convert categorical columns to numerical
orders_class = pd.get_dummies(orders_cluster)

In [82]:
#calculate and show correlations
corr_matrix = orders_class.corr()
corr_matrix['LTVCluster'].sort_values(ascending=False)

LTVCluster                                             1.000000
m6_Revenue                                             0.975310
customer_unique_id_35de25c73544f00e8adde226a4d1bdcc    0.166346
customer_unique_id_82fb46292a999e7db9e2391c407f76e4    0.166346
customer_unique_id_a0e0770d529c5da6dd71d5be8620aa3b    0.166346
                                                         ...   
customer_unique_id_861eff4711a542e4b93843c6dd7febb0   -0.000884
customer_unique_id_2e6a42a9b5cbb0da62988694f18ee295   -0.000884
Revenue                                               -0.003356
RevenueCluster                                        -0.010700
Recency                                               -0.022013
Name: LTVCluster, Length: 8523, dtype: float64

In [83]:
orders_class.head()

,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,m6_Revenue,LTVCluster,customer_unique_id_0000f46a3911fa3c0805444483337064,customer_unique_id_0005e1862207bf6ccc02e4228effd9a0,customer_unique_id_0011805441c0d1b68b48002f1d005526,customer_unique_id_00191a9719ef48ebb5860b130347bf33,customer_unique_id_001f3c4211216384d5fe59b041ce1461,customer_unique_id_002043098f10ba39a4600b6c52fbfe3c,customer_unique_id_002d71b244beb91ca7030b15ab526446,customer_unique_id_0036a074f98b80c4f1fc33dbbcf9c552,customer_unique_id_0037d2bbb9ebc39d9114aea27ee16d72,customer_unique_id_003e28cb6fab8a0901bff9229612375f,customer_unique_id_0043d65f8039b4a4d43bc7b0f450ac2c,customer_unique_id_00446a03d7a34d191507c9e42870d0a5,customer_unique_id_0049db44bfd6cee26ac9c649329ff6ea,customer_unique_id_00522457bba00bba576a34fd9b4808fd,customer_unique_id_0055b522bc89cbfbf376d8b59a628470,customer_unique_id_007953b3182d5ccc9189a694f5c68163,customer_unique_id_00842c7d0cedb9e1edad8e9b7d148f09,customer_unique_id_008fb2ecc75561d202dd4db2e0ceec25,customer_unique_id_00941614a654458b888b8780815bd26c,customer_unique_id_0096c18412349537abc45e350581cbbc,customer_unique_id_0098f4847541cc7d676f9a5efc64c0f0,customer_unique_id_00a218ea82c458456098caa1c4a7e796,customer_unique_id_00a7d14cb10e11b7f7d65120a7e3942f,customer_unique_id_00ac9cd5c4ad19e16e7c6f6864711737,customer_unique_id_00ad5165112e0e9a91a23b1132716584,customer_unique_id_00c003ed13c76042b78d53fbc26930d0,customer_unique_id_00c3624c55261d9ce593f30e3815a880,customer_unique_id_00cc12a6d8b578b8ebd21ea4e2ae8b27,customer_unique_id_00cc7c54469f4177319d6bb12e883652,customer_unique_id_00cc8551c0621bc65a440e836bb652d7,customer_unique_id_00dd4390a8e8ad7a126e8331decc49ce,customer_unique_id_00e707efa5ad9f0e6bcaeeb87910693b,...,customer_unique_id_ff12bd0a2ed9674fdcad62ae10fcb909,customer_unique_id_ff136252a2fd806ce234d85a09afbaef,customer_unique_id_ff253dd048167b1feddeb431db05d3d5,customer_unique_id_ff2cfbe44d7249b98eed0e860fe3e53c,customer_unique_id_ff4050535c1351cd0b834ccf1c9d07f1,customer_unique_id_ff4159b92c40ebe40454e3e6a7c35ed6,customer_unique_id_ff44401d0d8f5b9c54a47374eb48c1b8,customer_unique_id_ff49569c47039afb2424f276d9071368,customer_unique_id_ff4b2772a9368e8b30a1ebd484de12c9,customer_unique_id_ff5017601705e932d5545e0eb337cd0b,customer_unique_id_ff51a896842f2bdced342631e2699d49,customer_unique_id_ff57311a2fb5f9aa3477cc13c1c62a50,customer_unique_id_ff580ab4901e3619925a27db44eec095,customer_unique_id_ff5d0b841838e762e18145556c73bf01,customer_unique_id_ff6275fe63f74d9a63ce29460d90dfac,customer_unique_id_ff64dc057067aecad9bf2c94b272e90a,customer_unique_id_ff6c80eff6bc50a0c3b10abd65847a2a,customer_unique_id_ff6eae41d4cbdc5af3844a39a89431e6,customer_unique_id_ff7014eeb94589da99e822e2618090fa,customer_unique_id_ff70aa1b53f643263ead50eed3166164,customer_unique_id_ff86fefe11696b1e957173516339bcd8,customer_unique_id_ff8892f7c26aa0446da53d01b18df463,customer_unique_id_ff88dcb0fdf8854d0eadfa10ea548aa2,customer_unique_id_ff8d27e48d6492ebb97e3052142c47bf,customer_unique_id_ff959aedd362b48bb97c0abeb0b36b62,customer_unique_id_ff9b12e78f7c6c8ad5327b63374c370c,customer_unique_id_ff9bc66daa6702543b16f3c4897e300b,customer_unique_id_ff9ff869636c26565ded16038be5fa7d,customer_unique_id_ffa17ff63f95ea3c838670e61d2666e0,customer_unique_id_ffaa5b5559566bb4dbf71d554310e760,customer_unique_id_ffafb9f09293653c2486abe52e5e05a4,customer_unique_id_ffbb866d7c0d272f9fe12de1b9ee9173,customer_unique_id_ffd44de1326179fff550dae07c6884c2,customer_unique_id_ffe254cc039740e17dd15a5305035928,customer_unique_id_ffe2dd1f3b0cbf0b5f35e818ec03c49b,customer_unique_id_ffe3e199b9d0b7fb7d2d29a5b9498447,customer_unique_id_ffe4c3aa14d4702fb11fe5b3c4dd755c,customer_unique_id_ffedff0547d809c90c05c2691c51f9b7,customer_unique_id_ffef0ffa736c7b3d9af741611089729b,customer_unique_id_ffffd2657e2aad2907e67c3e9daecbeb
0,15,3,1,0,124.99,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [84]:
#create X and y, X will be feature set and y is the label - LTV
X = orders_class.drop(['LTVCluster','m6_Revenue'],axis=1)
y = orders_class['LTVCluster']

In [85]:
#split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=27)

In [86]:
# XGBoost Multiclassification Model
ltv_xgb_model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1,objective= 'multi:softprob',n_jobs=-1).fit(X_train, y_train)

print('Accuracy of XGB classifier on training set: {:.2f}'
       .format(ltv_xgb_model.score(X_train, y_train)))
print('Accuracy of XGB classifier on test set: {:.2f}'
       .format(ltv_xgb_model.score(X_test[X_train.columns], y_test)))

y_pred = ltv_xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))

Accuracy of XGB classifier on training set: 0.99
Accuracy of XGB classifier on test set: 0.99
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       422
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         1

    accuracy                           0.99       426
   macro avg       0.33      0.33      0.33       426
weighted avg       0.98      0.99      0.99       426

